In [1]:
from tools import utils, config, trainer, parts
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from torch.nn import functional as F
from tools import utils

plt.style.use('fast')
PLOT_DIR = 'plots'

cfg = config.from_yaml("experiments\exp1_cmpe597_regular_mnist_robust30\config.yaml")
dataset = utils.load_dataset_module(**cfg.data_supervised)
dataset.torch_seed()
test_loader = dataset.get_test_loader(**cfg.data_supervised)
test_dataset = dataset.get_test_dataset()

def get_model(cfg, model_path):
    model = utils.load_model(**cfg.model)
    model.load_state_dict(torch.load(model_path))
    part_manager = parts.PartManager(model)
    part_manager.enable_all()
    trn = trainer.ModelTrainer(model, cfg.trainer_sup, part_manager)
    model.eval();
    return model, trn, part_manager

model1, trn1, manager1 = get_model(cfg, "experiments\exp1_cmpe597_regular_mnist_robust30\checkpoint.pth")
model2, trn2, manager2 = get_model(cfg, "experiments\exp1_cmpe597_regular_mnist_robust50\checkpoint.pth")
model3, trn3, manager3 = get_model(cfg, "experiments\exp1_cmpe597_regular_mnist_robust70\checkpoint.pth")

Loading dataset: data\mnist.py
Created 4 parts.
Enable all parts
checkpoint to cuda:0
Created 4 parts.
Enable all parts
checkpoint to cuda:0
Created 4 parts.
Enable all parts
checkpoint to cuda:0


### Get samples for each class

In [2]:
classes = [3, 8]
n_classes = len(classes)
class_size = 500
classes_to_accumulate = [c for c in classes]
examples = {i: [] for i in classes}

i = 0
while i < len(test_dataset):
    x, y = test_dataset[i]
    if not y in classes:
        i += 1
        continue
    
    if not y in classes_to_accumulate:
        i+=1
        continue
    
    
    examples[y].append(i)
    
    if len(examples[y]) == class_size:
        classes_to_accumulate.remove(y)
        if len(classes_to_accumulate) == 0:
            break
    i+=1
    
[f'{k}:{len(v)}' for k, v in examples.items()]

['3:500', '8:500']

### Get Outputs for each model and class

In [3]:
def get_activations(model, trn, part_manager, part_i):
    part_output_list = [[] for _ in range(n_classes)]

    for class_i, class_examples in examples.items():
        class_order = classes.index(class_i)
        for example_i in class_examples:
            x, y = test_dataset[example_i]
            x = x.unsqueeze(0).to(trn.device)
            _ = model(x)
            
            part_output = getattr(part_manager.parts[part_i].get_loss_end_layer(), trainer.SAVED_OUTPUT_NAME)
            part_output = torch.squeeze(part_output).cpu().detach().numpy()
            part_output_list[class_order].append(part_output)

    activations = np.array(part_output_list)
    return activations

part_i = 1
activations1 = get_activations(model1, trn1, manager1, part_i)
activations2 = get_activations(model2, trn2, manager2, part_i)
activations3 = get_activations(model3, trn3, manager3, part_i)

d:\Miniconda3\envs\env\lib\site-packages\torch\nn\modules\module.py:1102: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return forward_call(*input, **kwargs)


In [4]:
manager1.parts

[loss_start_i=0
 loss_end_i=2
 layers=
 0- Conv2d(1, 20, kernel_size=(3, 3), stride=(1, 1))
 1- BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 2- ReLU()
 =====================================,
 loss_start_i=1
 loss_end_i=3
 layers=
 0- MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 1- Conv2d(20, 5, kernel_size=(1, 1), stride=(1, 1))
 2- BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 3- ReLU()
 =====================================,
 loss_start_i=2
 loss_end_i=3
 layers=
 0- MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 1- Flatten(start_dim=1, end_dim=-1)
 2- Linear(in_features=180, out_features=40, bias=True)
 3- ReLU()
 =====================================,
 loss_start_i=0
 loss_end_i=0
 layers=
 0- Linear(in_features=40, out_features=10, bias=True)
 1- LogSoftmax(dim=None)
 =====================================]

In [5]:
def mean_class_difference(act):
    return np.square((act[0].mean(axis=1) - act[1].mean(axis=1))).mean()

In [6]:
mean_class_difference(activations1)


0.08495415

In [7]:
mean_class_difference(activations2)

0.115243375

In [8]:
mean_class_difference(activations3)

0.13327244

Save activations

In [9]:
np.save(r"D:\projects\adversarial_robustness\experiments\exp1_cmpe597_regular_mnist_robust30\activations.npy", activations1)
np.save(r"D:\projects\adversarial_robustness\experiments\exp1_cmpe597_regular_mnist_robust50\activations.npy", activations2)
np.save(r"D:\projects\adversarial_robustness\experiments\exp1_cmpe597_regular_mnist_robust70\activations.npy", activations3)